In [ ]:
# Importing all necessary libraries
import cv2
import os

In [ ]:
# Read the video from specified path
cam = cv2.VideoCapture("/media/ppl3.mp4")

In [ ]:
try:
	
	# creating a folder named data
	if not os.path.exists('data'):
		os.makedirs('data')

# if not created then raise error
except OSError:
	print ('Error: Creating directory of data')

# frame
currentframe = 0

while(True):
	
	# reading from frame
	ret,frame = cam.read()

	if ret:
		# if video is still left continue creating images
		name = './data/frame' + str(currentframe) + '.png'
		print ('Creating...' + name)

		# writing the extracted images
		cv2.imwrite(name, frame)

		# increasing counter so that it will
		# show how many frames are created
		currentframe += 1
	else:
		break

# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

Creating..../data/frame0.png
Creating..../data/frame1.png
Creating..../data/frame2.png
Creating..../data/frame3.png
Creating..../data/frame4.png
Creating..../data/frame5.png
Creating..../data/frame6.png
Creating..../data/frame7.png
Creating..../data/frame8.png
Creating..../data/frame9.png
Creating..../data/frame10.png
Creating..../data/frame11.png
Creating..../data/frame12.png
Creating..../data/frame13.png
Creating..../data/frame14.png
Creating..../data/frame15.png
Creating..../data/frame16.png
Creating..../data/frame17.png
Creating..../data/frame18.png
Creating..../data/frame19.png
Creating..../data/frame20.png
Creating..../data/frame21.png
Creating..../data/frame22.png
Creating..../data/frame23.png
Creating..../data/frame24.png
Creating..../data/frame25.png
Creating..../data/frame26.png
Creating..../data/frame27.png
Creating..../data/frame28.png
Creating..../data/frame29.png
Creating..../data/frame30.png
Creating..../data/frame31.png
Creating..../data/frame32.png
Creating..../data/fr

In [ ]:
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm

In [ ]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r") 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [ ]:
def LogisticEncryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    L = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1
    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    C = round((L*L_y*10**4) % 256)
    C_r = round((L*L_y*10**4) % 256)
    C_g = round((L*L_y*10**4) % 256)
    C_b = round((L*L_y*10**4) % 256)
    x = 4*(S_x)*(1-S_x)
    y = 4*(S_y)*(1-S_y)
    imageMatrix,dimension, color = getImageMatrix(imageName)
    LogisticEncryptionIm = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            while x <0.8 and x > 0.2 :
                x = 4*x*(1-x)
            while y <0.8 and y > 0.2 :
                y = 4*y*(1-y)
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                C_r =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][0]) % N) ^ ((C_r + key_list[7]) % N)
                C_g =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][1]) % N) ^ ((C_g + key_list[7]) % N)
                C_b =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][2]) % N) ^ ((C_b + key_list[7]) % N)
                row.append((C_r,C_g,C_b))
                C = C_r
            else:
                C = ((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j]) % N) ^ ((C + key_list[7]) % N)
                row.append(C)
            x = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            y = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        LogisticEncryptionIm.append(row)
    im = Image.new('L', (dimension, dimension))
    if color:
        im = Image.new('RGB', (dimension, dimension))
    else: 
        im = Image.new('L', (dimension, dimension)) # L is for Black and white pixels    
    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = LogisticEncryptionIm[x][y]
    im.save(imageName, "BMP")

In [ ]:
image = "/media/temp"
ext = ".bmp"
key = "abcdefghijklm"

In [ ]:
folder_dir = "/content/data"
# cropped_images = []
for images in os.listdir(folder_dir):
  face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
  img = cv2.imread('/content/data/'+images)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.1, 4)
  print(images)
  for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.imwrite('/media/face detection result.png', img)
  for (x, y, w, h) in faces:
    cropped_image = img[y:y+h, x:x+w]
    cv2.imwrite('/media/temp.bmp', cropped_image)
    LogisticEncryption(image + ext, key)
    im = Image.open(image + ".bmp", 'r')
    LogisticResult = cv2.imread("/media/temp.bmp",cv2.IMREAD_COLOR)
    img[y:y+h, x:x+w] = LogisticResult
    cv2.imwrite('/media/encrypted_frames/EncryptionResult'+images, img)
    # cropped_images.append(img)

frame20.png
frame36.png
frame13.png
frame15.png
frame1.png
frame29.png
frame24.png
frame14.png
frame49.png
frame47.png
frame61.png
frame51.png
frame26.png
frame43.png
frame44.png
frame17.png
frame40.png
frame32.png
frame58.png
frame27.png
frame16.png
frame34.png
frame33.png
frame63.png
frame8.png
frame6.png
frame28.png
frame23.png
frame54.png
frame39.png
frame46.png
frame52.png
frame60.png
frame10.png
frame59.png
frame4.png
frame19.png
frame53.png
frame30.png
frame31.png
frame64.png
frame22.png
frame2.png
frame38.png
frame50.png
frame45.png
frame25.png
frame57.png
frame3.png
frame62.png
frame21.png
frame0.png
frame7.png
frame35.png
frame5.png
frame48.png
frame42.png
frame12.png
frame9.png
frame41.png
frame11.png
frame18.png
frame37.png
frame55.png
frame56.png


In [ ]:
# importing libraries
import os
import cv2
from PIL import Image
import os.path
import shutil
# Checking the current directory path
print(os.getcwd())

# Folder which contains all the images
# from which video is to be generated
os.chdir("/media/encrypted_frames")
path = "/media/encrypted_frames"

mean_height = 0
mean_width = 0
num_of_images = len(os.listdir('.'))
print(num_of_images)

for file in os.listdir('.'):
	im = Image.open(os.path.join(path, file))
	width, height = im.size
	mean_width += width
	mean_height += height
	# im.show() # uncomment this for displaying the image

# Finding the mean height and width of all images.
# This is required because the video frame needs
# to be set with same width and height. Otherwise
# images not equal to that width height will not get
# embedded into the video
mean_width = int(mean_width / num_of_images)
mean_height = int(mean_height / num_of_images)

# print(mean_height)
# print(mean_width)

# Resizing of the images to give
# them same width and height
for file in os.listdir('.'):
	if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith("png"):
		# opening image using PIL Image
		im = Image.open(os.path.join(path, file))

		# im.size includes the height and width of image
		width, height = im.size
		print(width, height)

		# resizing
		imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)
		imResize.save( file, 'JPEG', quality = 95) # setting quality
		# printing each resized image name
		print(im.filename.split('\\')[-1], " is resized")


# Video Generating function
def generate_video():
	image_folder = '/media/encrypted_frames' # make sure to use your folder
	video_name = 'encryptedvideo.avi'
	os.chdir("/media")
	
	images = [img for img in os.listdir(image_folder)
			if img.endswith(".jpg") or
				img.endswith(".jpeg") or
				img.endswith("png")]
	
	# Array images should only consider
	# the image files ignoring others if any
	print(images)

	frame = cv2.imread(os.path.join(image_folder, images[0]))

	# setting the frame width, height width
	# the width, height of first image
	height, width, layers = frame.shape

	video = cv2.VideoWriter(video_name, 0, 1, (width, height))

	# Appending the images to the video one by one
	for image in images:
		video.write(cv2.imread(os.path.join(image_folder, image)))
	
	# Deallocating memories taken for window creation
	cv2.destroyAllWindows()
	video.release() # releasing the video generated


# Calling the generate_video function
generate_video()


/media
EncryptionResultframe14.png
EncryptionResultframe17.png
EncryptionResultframe50.png
EncryptionResultframe34.png
EncryptionResultframe45.png
EncryptionResultframe37.png
EncryptionResultframe52.png
EncryptionResultframe61.png
EncryptionResultframe58.png
EncryptionResultframe12.png
EncryptionResultframe35.png
EncryptionResultframe54.png
EncryptionResultframe8.png
EncryptionResultframe20.png
EncryptionResultframe63.png
EncryptionResultframe26.png
EncryptionResultframe46.png
EncryptionResultframe11.png
EncryptionResultframe29.png
EncryptionResultframe28.png
EncryptionResultframe21.png
EncryptionResultframe33.png
EncryptionResultframe41.png
EncryptionResultframe22.png
EncryptionResultframe60.png
EncryptionResultframe31.png
EncryptionResultframe51.png
EncryptionResultframe16.png
EncryptionResultframe57.png
EncryptionResultframe15.png
EncryptionResultframe4.png
EncryptionResultframe24.png
EncryptionResultframe18.png
EncryptionResultframe49.png
EncryptionResultframe42.png
EncryptionResul

In [ ]:
# for (x, y, w, h) in faces:
#     cropped_image = img[y:y+h, x:x+w]
#     cv2.imwrite('/img/temp.bmp', cropped_image)
#     LogisticEncryption(image + ext, key)
#     im = Image.open(image + ".bmp", 'r')
#     LogisticResult = cv2.imread("/img/temp.bmp",cv2.IMREAD_COLOR)
#     img[y:y+h, x:x+w] = LogisticResult

In [ ]:
# cv2.imwrite('/img/Encryption Result.bmp', img)

True